# SCF algorithms

As we discussed in [2_preconditioning.ipynb](/notebooks/2_preconditioning.ipynb) the self-consistent field procedure required to solve the DFT problem can be written as a fixed-point problem
$$ F(\rho) = \rho $$
where $F$ represents the basic SCF step. That is the construction of the Kohn-Sham Hamiltonian $H(\rho)$ given the density $\rho$, followed its diagonalisation to obtain its eigenpairs $(\varepsilon_{k i}, \psi_{ki})$
and from these a new density
$$ \rho(r) = \sum_{k\in\text{BZ}} \sum_i f\left(\frac{\varepsilon_{ki} - \varepsilon_F}{\theta}\right) \, \psi_{ki}(r) \, \psi_{ki}^\ast(r)$$
with the Fermi level $\varepsilon_F$ chosen to conserve the number of electrons.

In this exercise we will take the function $F$ for "granted" (i.e. delivered by DFTK) and we will investigate multiple algorthms to find the fixed-point density satisfying $F(\rho) = \rho $. Our setting is defined by the following function, which builds and discretises an LDA model for an aluminium supercell, see [2_preconditioning.ipynb](/notebooks/2_preconditioning.ipynb) for details.

In [ ]:
using DFTK
using ASEconvert

function aluminium_setup(repeat=1; Ecut=13.0, kgrid=(2, 2, 2))
    # Use ASE to make an aluminium supercell
    pysystem = ase.build.bulk("Al", cubic=true) * pytuple((repeat, 1, 1))
    pysystem.rattle()  # Displace atoms a little (makes problem harder)
    
    # Convert to AbstractSystem and attach pseudopotentials:
    aluminium = pyconvert(AbstractSystem, pysystem)
    system = attach_psp(aluminium; Al="hgh/lda/al-q3")

    # Construct an LDA model and discretise
    model = model_LDA(system; temperature=1e-3, smearing=Smearing.Gaussian())
    PlaneWaveBasis(model; Ecut, kgrid)
end

**(a) Fixed-point iterations.** The easiest way to solve $F(\rho) = \rho$ are fixed-point iterations, i.e.
$$ \rho_{n+1} = F(\rho_n), $$
starting from a hopefully good initial guess $\rho_0$. DFTK automatically provides a reasonable
guess density, such that we only need to take care of the iterations themselves.
In the language of DFTK this algorithm is written as:

In [ ]:
function fixed_point_iteration(F, ρ₀, maxiter; tol)
    # F:        The SCF step function
    # ρ₀:       The initial guess density
    # maxiter:  The maximal number of iterations to be performed
    # tol:      The selected convergence tolerance
    
    ρ  = ρ₀
    Fρ = F(ρ)
    for n = 1:maxiter 
        # If change less than tolerance, break iterations:
        if norm(Fρ - ρ) < tol
            break
        end
        ρ  = Fρ
        Fρ = F(ρ)
    end
    
    # Return some stuff DFTK needs ...
    (fixpoint=ρ, converged=norm(Fρ-ρ) < tol)
end;

# use this algorithm inside DFTK's SCF for solving the silicon problem
# (the other parameters are needed to overwrite some DFTK defaults we don't want to use yet).
self_consistent_field(aluminium_setup(1); solver=fixed_point_iteration, damping=1.0,
                                          maxiter=40, mixing=SimpleMixing());

As can be observed this algorithm is not very good and in fact even fails to converge albeit we are only looking at a very simple system.

This is a known limitation of this algorithm, which is why it is not used in practice. Instead one introduces a so-called damping parameter $\alpha$, which is given a value between $0$ and $1$. One now iterates as follows:
$$ \rho_{n+1} = \rho_{n} + \alpha (F(\rho_n) - \rho_n) $$
In other words the update $F(\rho_n) - \rho_n$ proposed in the $n$-th SCF step is not fully taken, but scaled-down by the damping $\alpha$.

Modify `fixed_point_iteration` such that it supports this *damped* fixed-point iteration. In other words implement damping *inside* your algorithm and not by changing the `damping` parameter of the `self_consistent_field` function driving the SCF.  
Using your algorithm try different values for $\alpha$ between $0$ and $1$ and estimate roughly the $\alpha$ which gives fastest convergence. For which $\alpha$ do you observe no convergence at all?

**Remark:** The observations you make here are general. One can proove that every SCF converges (locally) if a small enough $\alpha > 0$ is chosen.

**(b) Anderson acceleration.** The `fixed_point_iteration` function above (with the damping extension) actually already covers the main gist of the DFT algorithms used in practice. One additional idea to make things converge faster is Anderson acceleration, where not only $\rho_n$ and $F(\rho_n)$, but also older iterates are used to propose the next density.

For Anderson one exploits that the update $R(\rho) = F(\rho) - \rho$ is also the residual of the fixed-point problem $F(\rho) = \rho$, i.e. how far away we are from the fixed-point density. A good next density $\rho_{n+1}$ therefore should be found by minimising an approximation for $R(\rho_{n+1})$. Assuming the SCF was linear in the density (which it is not), a good idea is to find a linear combination of residuals
$$\min_{\beta_i} \left\| \sum_i \beta_i R(\rho_i) \right\|^2$$
which has the smallest possible norm and to use these coefficients $\beta_i$ to extrapolate the next
density
$$ \rho_{n+1} =  \sum_i \beta_i (\rho_i + \alpha R(\rho_i)) $$
where you notice the "standard" damped fixed-point iteration in the summed terms.

In terms of an algorithm this is

In [ ]:
function anderson_iteration(F, ρ₀, maxiter; tol)
    # F:        The SCF step function
    # ρ₀:       The initial guess density
    # maxiter:  The maximal number of iterations to be performed
    # tol:      The selected convergence tolerance
    
    converged = false
    ρ  = ρ₀
    ρs = []
    Rs = []
    for n = 1:maxiter
        Fρ = F(ρ)
        Rρ = Fρ - ρ
        converged = norm(Rρ) < tol
        converged && break
        
        ρnext = vec(ρ) .+ vec(Rρ)
        if !isempty(Rs)
            M = hcat(Rs...) .- vec(Rρ)
            βs = -(M \ vec(Rρ))
            
            for (iβ, β) in enumerate(βs)
                ρnext .+= β .* (ρs[iβ] .- vec(ρ) .+ Rs[iβ] .- vec(Rρ))
            end
        end
                    
        push!(ρs, vec(ρ))
        push!(Rs, vec(Rρ))
        ρ = reshape(ρnext, size(ρ₀)...)
    end
    
    # Return some stuff DFTK needs ...
    (fixpoint=ρ, converged=converged)
end

To work with this algorithm we will use DFTK's intrinsic mechanism to choose a damping. The syntax for this is

```julia
repeat = 1
self_consistent_field(aluminium_setup(repeat);
                      solver=anderson_iteration,
                      damping=0.8, maxiter=40,
                      mixing=SimpleMixing());
```
to choose a damping of $\alpha = 0.8$ and run for at most `maxiter` iterations.

(i) Based on this Anderson implementation verify (by making a few experiments) that the algorithm converges for `repeat=1` for any $0 < \alpha \leq 2$. You may now use the `damping` parameter for changing the value $\alpha$ used by the SCF. State the number of iterations and runtimes you observe.

(ii) Pick $\alpha = 0.8$ and make the problem harder by increasing `repeat` (e.g. `2`, `4`, `6`, `8`). Can you make Anderson fail to converge? What do you notice in terms of the number of iterations and runtimes?

**(c) Mixing methods.** Anderson allows us to push the boundary for SCF methods, but for larger or more challenging systems it is not fully sufficient. The next ingredient for a stable SCF procedure is based on the insight that the convergence properties of an SCF provably depend on the dielectric properties of materials, which is simulated. Amongst others this is to say that insulators (like glass), semiconductors (like silicon) or metals (like aluminium) have rather differing SCF behaviours. As a result the ideal SCF procedure should be slightly different for each material.

The standard approach to include material specificity into the SCF is to employ *preconditioned* damped fixed-point iterations.
To explain the idea, let us consider again a framework without Anderson acceleration.
Preconditioned iterations are then
$$ \rho_{n+1} = \rho_{n} + \alpha P^{-1} (F(\rho_n) - \rho_n), $$
where $P^{-1}$ is a preconditioner that hopefully improve convergence.
To re-introduce Anderson around this iteration
just replace the previous definition of $R$ by
$R(\rho) = P^{-1} (F(\rho_n) - \rho_n)$.

Finding a good preconditioner $P$ is not always easy and for some cases satisfactory options are not yet known. For our aluminium case, however, we are lucky. The `KerkerMixing` implemented in DFTK provides a good $P$ for aluminium.

You might wonder about the term *mixing*. In an interdisciplinary community like DFT, different scientists use different vocabulary and "mixing" is the physicists' term used for preconditioning.

To use `KerkerMixing` with DFTK run the SCF as follows
```julia
self_consistent_field(basis; damping=0.8, mixing=KerkerMixing());
```

Try this setup for different values of `repeat` and check the number of iterations needed. Other mixings DFTK has to offer are `DielectricMixing` (best for semiconductors), `SimpleMixing` (which is $P = I$, i.e. no preconditioner at all, best for insulators) or `LdosMixing` (self-adapting, suitable for both metals *or* insulators *or* inhomogeneous mixtures). Note that `LdosMixing` is the default in DFTK (i.e. used if the `mixing` parameter is *not* supplied to `self_consistent_field`. Try these mixings (`SimpleMixing`, `DielectricMixing`, `LdosMixing` and `KerkerMixing`) and summarise your findings.

You should notice that choosing a preconditioner matching the material under study aids a fast SCF convergence, but that sometimes being off does not seem to do much harm for our case. For larger values of `repeat` (beyond what you can probably effort on your laptop) this is no longer true and one needs to be very careful in selecting the right preconditioner. See for example the investigation in [this paper](https://michael-herbst.com/publications/2020.09.03_ldos_preconditioning.pdf). 